In [1]:
import calendar
import datetime
import pandas as pd

In [2]:
# enter the first day of the target month
targetMonth = datetime.date(2021, 3, 1)

In [6]:
# create a list of days in the target month
daysInMonth = calendar.monthrange(targetMonth.year, targetMonth.month)[1]
daysOfMonth = [datetime.date(targetMonth.year, targetMonth.month, day) for day in range(1, daysInMonth + 1)]

In [7]:
# create day of week labels for days in target month
dayOfWeek = [x.weekday() for x in daysOfMonth]
dayOfWeekChar = [calendar.day_name[x] for x in dayOfWeek]

In [137]:
# combine the days features as a dict, then convert that to a pandas dataframe
# repeat for each potential meal
mealsDict = {'dayOfMonth':dayOfMonth
             , 'dayOfWeek':dayOfWeek
             , 'dayOfWeekChar':dayOfWeekChar
            }

mealsDF = pd.concat([pd.DataFrame(mealsDict).assign(meal = 'breakfast')
                     , pd.DataFrame(mealsDict).assign(meal = 'lunch')
                     , pd.DataFrame(mealsDict).assign(meal = 'dinner')]
                   ).assign(whatToEat = None)

In [140]:
# write a function to assign meals to days
def assignMeal(targetDay, targetMeal, mealIdea):
    # verify that all inputs are entered in the appropriate formats
    if type(targetDay) is not list:
        print('Error - enter your target days as a list')
        return
    if type(targetMeal) is not list:
        print('Error - enter your target meals as a list')
        return
    if type(mealIdea) is not str:
        print('Error - enter your meal idea as a string')
        return
    
    # verify that all targetMeal values are strings and standard meal names
    nonStringMeals = sum([0 if type(x) is str else 1 for x in targetMeal])
    if nonStringMeals > 0:
        print('Error - enter all of your target meals as strings')
        return
        
    nonStandardMeals = sum([0 if x in ['breakfast', 'lunch', 'dinner'] else 1 for x in targetMeal])
    if nonStandardMeals > 0:
        print('Error - available meals are breakfast, lunch, and dinner')
        return
        
    # check how target days are identified and verify there's only one style used
    if len(set([type(x) for x in targetDay])) > 1:
        print('Error - enter all of your target days in the same format')
    
    targetDayType = type(targetDay[0])
    
    # assign a dataframe column
    if targetDayType is datetime.date:
        targetDayField = mealsDF['dayOfMonth']
    elif targetDayType is int:
        targetDayField = mealsDF['dayOfWeek']
    elif targetDayType is str:
        targetDayField = mealsDF['dayOfWeekChar']
        
    mealsDF['whatToEat'][targetDayField.isin(targetDay) & mealsDF['meal'].isin(targetMeal)] = mealIdea

C:\Users\evief\Anaconda\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,dayOfMonth,dayOfWeek,dayOfWeekChar,meal,whatToEat
0,2021-03-01,0,Monday,breakfast,fast
1,2021-03-02,1,Tuesday,breakfast,None
2,2021-03-03,2,Wednesday,breakfast,fast
3,2021-03-04,3,Thursday,breakfast,None
4,2021-03-05,4,Friday,breakfast,None
...,...,...,...,...,...
26,2021-03-27,5,Saturday,dinner,None
27,2021-03-28,6,Sunday,dinner,None
28,2021-03-29,0,Monday,dinner,None
29,2021-03-30,1,Tuesday,dinner,None


In [38]:
# start assigning meals!

# I prep by the week, so the first step is to mark out meals that are part of a week that began last month

# next I can label all my boring weekday breakfasts
breakfast = [b if d in ['Saturday', 'Sunday'] else 'oatmeal' for b, d in zip(breakfast, dayOfWeekChar)]

# and treat myself to dinner out on Friday
dinner = ['go out to eat' if d == 'Friday' else din for din, d in zip(dinner, dayOfWeekChar)]

# 5:2 life makes Monday and Wednesday breakfast and lunches easy too
breakfast = ['fast' if d in ['Monday', 'Wednesday'] else b for b, d in zip(breakfast, dayOfWeekChar)]
lunch = ['fast' if d in ['Monday', 'Wednesday'] else l for l, d in zip(lunch, dayOfWeekChar)]